In [ ]:
import os
import sys

sys.path = [
    '/kaggle/input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master',
] + sys.path

import torch
os.listdir("/kaggle/input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master")

from efficientnet_pytorch import EfficientNet

In [ ]:
from efficientnet_pytorch import EfficientNet
# Imports here
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import pandas as pd
import os
import random
import math
import skimage.io
#from csv_loader import load_csv

# Tiff visualisation imports and downloads
import numpy as np
import tifffile as tiff

# For re-importing python modules
import importlib

In [ ]:
# #use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [ ]:
class load_csv(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)# todo remove sample for debug
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.annotations)
        
    
    def __getitem__(self, index):
        #img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image_id = self.annotations.iloc[index, 0]
        img_path = os.path.join(self.root_dir, str(image_id) +".tiff")
        img = skimage.io.MultiImage(img_path)[2]
        
        image = torch.from_numpy(img).permute(2,0,1).float()
        
        #Image.MAX_IMAGE_PIXELS = None
                
        #image.transform = transforms.RandomResizedCrop(224)
        
        #y_label = torch.tensor(int(self.annotations.iloc[index,:]['isup_grade']))
        #isup_grade = int(self.annotations.iloc[index,:]['isup_grade'])
        
        #label = np.zeros(6).astype(np.float32)
        #y_label = label[isup_grade] = 1.
        #y_label = torch.tensor(y_label)
        
        self.transform= transforms.Compose([transforms.ToPILImage(),
                                            transforms.Resize((int(1840/2),int(1728/2))), 
                                            transforms.ToTensor()])
                                            #transforms.Normalize([0.485, 0.456, 0.406],
                                                             #      [0.229, 0.224, 0.225])])
        if self.transform:
            image = self.transform(image)
        
        return (image, np.array(0))#1840, 1728

In [ ]:
#criterion = nn.NLLLoss()

#model = EfficientNet.from_pretrained('efficientnet-b4', num_classes=6)

model = EfficientNet.from_name('efficientnet-b4')
model.load_state_dict(torch.load("/kaggle/input/efficientnet-pytorch/efficientnet-b4-e116e8b3.pth"))


model._fc = nn.Sequential(nn.Linear(model._fc.in_features, 6),
                          nn.LogSoftmax(dim=1))

#optimizer = optim.Adam(model.parameters(), lr=0.001)

checkpoint = torch.load("/kaggle/input/cnnmodelv1/model.pth")
model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#epoch = checkpoint['epoch']
#loss = checkpoint['loss']

In [ ]:
# def validate_data_function(model, test_loader, criterion):
#     test_loss = 0
#     accuracy = 0
    
#     for ii, (inputs, labels, image_id) in enumerate(test_loader):
        
#         inputs, labels = inputs.to(device), labels.to(device)
        
#         output = model.forward(inputs)
#         test_loss += criterion(output,labels.long())#.item()
        
#         #ps = torch.exp(output)
#         #equality = (labels.argmax(dim=1) == output.argmax(dim=1))
#         equality = (labels == output.argmax(dim=1))
#         accuracy += equality.type(torch.FloatTensor).mean()
#         #pred = output.cpu().data.numpy().argmax()
#         #qwk = cohen_kappa_score(pred, labels, weights='quadratic')
    
#     return test_loss, accuracy, image_id

In [ ]:
# model.eval()
    
# with torch.no_grad():
#     test_loss, accuracy, image_id = validate_data_function(model, train_loader, criterion)
                
# print("Test Accuracy: {}%".format(accuracy*100/len(train_loader)))


In [ ]:
#sampleimg=next(iter(load_csv(csv_file=csv_file_dir, root_dir=submission_test_path)))

In [ ]:
sampleimg[0].shape

In [ ]:
# Creating submission file

# For testing
#######Comment before submission
# file_info = pd.read_csv('../input/prostate-cancer-grade-assessment/train.csv').copy()
# sample_size = 5
# submit_list = file_info.sample(sample_size)
# submit_list.to_csv("sample.csv", sep=",", index=False)
# submission_test_path = "../input/prostate-cancer-grade-assessment/train_images/"
# #csv_file_dir = './sample.csv'
# csv_file_dir = '/kaggle/input/prostate-cancer-grade-assessment/train.csv'
###########

# For submission
submission_test_path = "/kaggle/input/prostate-cancer-grade-assessment/test_images/"
csv_file_dir = '/kaggle/input/prostate-cancer-grade-assessment/test.csv'

pred_y_excel = []
submission=None
if os.path.exists(submission_test_path):
    #print("dir found")
    test_df = pd.read_csv(csv_file_dir)
    test_data = load_csv(csv_file=csv_file_dir, root_dir=submission_test_path)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=False)
    #model.eval()
    dummy=0
    for ii2, (inputs2, labels2) in enumerate(test_loader):
        dummy=1+1
        inputs2, labels2 = inputs2.to(device), labels2.to(device)
    
        output2 = model.forward(inputs2)
        pred_y2 = output2.argmax(dim=1)

        pred_y_excel.append(int(pred_y2))
    #print("writing csv")
    #pred_y_excel = [3 for i in range(test_df.shape[0])]
    submission = pd.DataFrame({'image_id':test_df.image_id, 'isup_grade':pred_y_excel})
    submission.to_csv("submission.csv", sep=",", index=False)
else:
    #print("dir not found, building dummy")
    test_df = pd.read_csv("/kaggle/input/prostate-cancer-grade-assessment/sample_submission.csv")
#     test_df['isup_grade'] = 0
#     fake_submission=test_df[['image_id','isup_grade']]
    test_df.to_csv('submission.csv', index=False)
    #print('submission saved')
    #print(fake_submission)